In [1]:
from ctmnet2.simulation import Simulator

## 设置输入输出的路段

In [2]:
U=[1,2,3,4]
D=[5,6,7,8]

## 建立路网、建立默认的信号灯、设置到达离去

In [56]:
from ctmnet2.networkcreation import create_from_xml

sim = create_from_xml("NET-BJH.xml")

for sec_id in sim.sections.keys():
    sim.create('SignalController',sec_id,{'cycle': 60})
    sim.create('Phase',sec_id,{'green_start': 20,'green_end': 59})
    sim.add_lamp(sec_id,sec_id,sec_id)
for u in U:
    sim.set_arrival(str(u),1900)
for d in D:
    sim.set_departure(str(d),1700)

## 开始仿真并输出结果

In [57]:
%%time

import numpy as np

result = []
r = []
for i in range(1800):
    r = []
    sim.run_single_step()
    V = sim.get_volume('cell')
    '''if i>1500:
        for d in D:
            sim.set_departure(str(d),900)
    if i>1650:
        for d in D:
            sim.set_departure(str(d),1750*2)'''
    if i>1500 and i<1800:
        for j in range(1,5):
            T=[]
            for k in range(len(V[str(j)])):
                T.append(V[str(j)][k]+V[str(j+4)][len(V[str(j)])-k-1])
                #print(j,k)
            r=r+[round(t,1) for t in T]
            #print('ok')
        result.append(r)

Wall time: 430 ms


查看每一个路段上的元胞数量

In [58]:
import numpy as np

sim.run_single_step()
V = sim.get_volume('cell')
n = 0
for j in range(1,9):
    n = n+len(V[str(j)])
    print(j,len(V[str(j)]),n)
    #t=np.array(V[str(j)])+np.array(V[str(j+30)])
    #print(t)

1 7 7
2 6 13
3 7 20
4 7 27
5 7 34
6 6 40
7 7 47
8 7 54


转置并查看元胞总数和时长

In [59]:
T = []
for i in range(len(result[0])):  # 行数
    t = []
    for j in range(len(result)):
        t.append(result[j][i])
    T.append(t)
#print(T)
print(len(T),len(T[0]))

27 299


仿真结果写入文本

In [60]:
with open('4Netdata-BJH.txt','a') as file_handle:
    for t in T:
        file_handle.write(str(t)+',\n')

计算路网中各路段元胞间坐标的平均值

In [35]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse("NET-BJH.xml")
root = tree.getroot()
para = 'coordinates'
coordinates=[]
for sec in root.iter('section'):
    sec_id = sec.attrib['id']
    print(sec_id)
    coors = re.findall(r"(\d+\.?\d*,\d+\.?\d*)",sec.find(para).text)
    COORS = []
    COOR = []
    COORDINATE = [0,0]
    COORDINATES = []
    for coor in coors:
        COOR = coor.split(',')
        COOR[0]=float(COOR[0])
        COOR[1]=float(COOR[1])
        COORS.append(COOR)
    #print(len(COORS))
    #print(COORS[6])
    for i in range(len(COORS)-1):
        COORDINATE[0] = (COORS[i][0]+COORS[i+1][0])/2
        COORDINATE[1] = (COORS[i][1]+COORS[i+1][1])/2
        COORDINATES.append(COORDINATE.copy())
    print(COORDINATES)
    coordinates.extend(COORDINATES)
    #print(sec_id,len(COORDINATES))

1
[[31.9296045, 118.821481]]
2
[[31.92682, 118.821803]]
3
[[31.928199499999998, 118.81975349999999]]
4
[[31.9280605, 118.8231735]]
5
[[31.9296045, 118.821481]]
6
[[31.92682, 118.821803]]
7
[[31.928199499999998, 118.81975349999999]]
8
[[31.9280605, 118.8231735]]


In [34]:
#print(len(coordinates))
print(coordinates)

[[31.9296045, 118.821481], [31.92682, 118.821803], [31.928199499999998, 118.81975349999999], [31.9280605, 118.8231735], [31.9296045, 118.821481], [31.92682, 118.821803], [31.928199499999998, 118.81975349999999], [31.9280605, 118.8231735]]


In [118]:
with open('Coordata-BJH.txt','a') as file_handle:
    for coor in coordinates:
        file_handle.write('{"lat":'+str(coor[0])+',"lng":'+str(coor[1])+'},\n')

## 区域内道路总长

In [37]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse("NET-BJH.xml")
root = tree.getroot()
para = 'lanes_length'
S = 0
for sec in root.iter('section'):
    S += float(sec.find(para).text)
print(round(S,2))

2379.58
